<a href="https://colab.research.google.com/github/OtolaHan/Otola_Han_KAAN/blob/main/L_2_data_mining_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная работа №2

Выполните эту ячейку, чтобы вернуться у знакомым данным из первой работы

In [1]:
# categories
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']
# obtain the documents containing the categories provided
from sklearn.datasets import fetch_20newsgroups

twenty_train = fetch_20newsgroups(subset='train', categories=categories, \
                                  shuffle=True, random_state=42)
import nltk



def format_rows(docs):
    """ format the text field and strip special characters """
    D = []
    for d in docs.data:
        temp_d = " ".join(d.split("\n")).strip('\n\t')
        D.append([temp_d])
    return D

def format_labels(target, docs):
    """ format the labels """
    return docs.target_names[target]

def check_missing_values(row):
    """ functions that check and verifies if there are missing values in dataframe """
    counter = 0
    for element in row:
        if element == True:
            counter+=1
    return ("The amoung of missing records is: ", counter)

def tokenize_text(text, remove_stopwords=False):
    """
    Tokenize text using the nltk library
    """
    tokens = []
    for d in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(d, language='english'):
            # filters here
            tokens.append(word)
    return tokens
import pandas as pd

# construct dataframe from a list
X = pd.DataFrame.from_records(format_rows(twenty_train), columns= ['text'])
# add category to the dataframe
X['category'] = twenty_train.target
# add category label also
X['category_name'] = X.category.apply(lambda t: format_labels(t, twenty_train))

## 4. Data Mining с использованием Pandas

Давайте займемся серьезной работой. Давайте научимся программировать некоторые идеи и концепции, изученные до сих пор в курсе Data Mining. Это единственный способ убедиться в истинной мощи фреймов данных Pandas.

### 4.1 Пропущенные значения

Во-первых, давайте рассмотрим случай, когда в нашем наборе данных есть *пропущенные значения*, и мы хотим удалить эти значения. В своем текущем состоянии наш набор данных не имеет пропущенных значений, но для практики мы добавим несколько записей с пропущенными значениями, а затем напишем некоторый код для работы с этими объектами, содержащими пропущенные значения. Вы сами увидите, как легко работать с пропущенными значениями, как только ваши данные будут преобразованы во фрейм данных Pandas.

Прежде чем перейти к кодированию, давайте быстро повторим то, что мы узнали на курсе Data Mining. В частности, давайте рассмотрим методы, используемые для работы с пропущенными значениями.

Наиболее распространенные причины наличия пропущенных значений в наборах данных связаны с тем, как данные были первоначально собраны. Хорошим примером этого является случай, когда пациент поступает в отделение неотложной помощи, данные собираются как можно быстрее, и в зависимости от состояния пациентов собираемые личные данные являются либо неполными, либо частично полными. В первом и последнем случаях мы сталкиваемся со случаем «пропущенных значений». Зная, что данные пациентов особенно важны и могут использоваться органами здравоохранения для проведения интересного анализа, мы, как специалисты по интеллектуальному анализу данных, сталкиваемся с трудной задачей: решить, что делать с этими пропущенными и неполными записями. Нам нужно работать с этими записями, потому что они определенно повлияют на наш анализ или алгоритмы обучения. Итак, что нам делать? Есть несколько способов обработки пропущенных значений, и некоторые из наиболее эффективных способов представлены ниже (Примечание: вы можете обратиться к слайдам - ​​Session 1 Handout для получения дополнительной информации).

- **Удаление объектов данных** - Здесь мы полностью отбрасываем записи, как только они содержат некоторые пропущенные значения. Это самый простой подход, и мы будем использовать его в этой тетради. Непосредственным недостатком этого подхода является то, что вы теряете некоторую информацию, а в некоторых случаях и слишком много. Теперь представьте, что половина записей имеет хотя бы одно или несколько пропущенных значений. Здесь перед вами стоит трудное решение: количество против качества. В любом случае, это решение нужно принимать осторожно, поэтому здесь в этой тетради на этом делается акцент.

- **Оценка пропущенных значений** - Здесь мы пытаемся оценить пропущенные значения на основе некоторых критериев. Хотя этот подход может оказаться эффективным, это не всегда так, особенно когда мы имеем дело с конфиденциальными данными, такими как **Пол** или **Имена**. Для таких полей, как **Адрес**, могут быть способы получить эти недостающие адреса с помощью некоторой техники агрегирования данных или получить информацию непосредственно из других баз данных или общедоступных источников данных.

- **Игнорировать пропущенное значение во время анализа** - Здесь мы просто игнорируем пропущенные значения и продолжаем наш анализ. Хотя это самый наивный способ обработки пропущенных значений, он может оказаться эффективным, особенно когда пропущенные значения содержат информацию, не важную для проводимого анализа. Но подумайте об этом некоторое время. Стали бы вы игнорировать пропущенные значения, особенно когда в наши дни трудно получить высококачественные наборы данных? Опять же, есть некоторые компромиссы, о которых мы поговорим позже в этой тетради.

- **Замена всеми возможными значениями** - Как эффективный и ответственный специалист по интеллектуальному анализу данных, нам иногда просто нужно приложить немало усилий и найти способы компенсировать эти пропущенные значения. Этот последний вариант - очень разумный вариант для случаев, когда данных не хватает (что почти всегда бывает) или когда речь идет о конфиденциальных данных. Представьте, что в нашем наборе данных есть поле **Возраст**, которое содержит много пропущенных значений. Поскольку **Возраст** - это непрерывная переменная, это означает, что мы можем построить отдельную модель для вычисления возраста для неполных записей на основе некоторого подхода, основанного на правилах, или вероятностного подхода.

Как упоминалось ранее, мы собираемся выбрать первый вариант, но вас могут попросить вычислить пропущенные значения, используя другой подход, в качестве упражнения. Давайте приступим!

Сначала мы хотим добавить фиктивные записи с пропущенными значениями, так как набор данных, который у нас есть, идеально составлен и очищен и не содержит пропущенных значений. Сначала давайте сами убедимся, что набор данных действительно не содержит пропущенных значений. Мы можем легко сделать это с помощью следующей встроенной функции, предоставляемой Pandas.

In [2]:
X.isnull()

,text,category,category_name
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
...,...,...,...
2252,False,False,False
2253,False,False,False
2254,False,False,False
2255,False,False,False


Функция `isnull` просматривает весь набор данных на наличие нулевых значений и возвращает `True` везде, где находит какое-либо отсутствующее поле или запись. Как вы увидите выше, и как мы и ожидали, наш набор данных выглядит чистым, и все значения присутствуют, поскольку `isnull` возвращает **False** для всех полей и записей. Но давайте начнем пачкать руки и построим хорошую небольшую функцию для проверки каждой из записей, столбец за столбцом, и возвращать приятное небольшое сообщение, сообщающее нам количество найденных отсутствующих записей. Это упражнение также побудит нас изучить другие возможности фреймов данных pandas. В большинстве случаев встроенных функций достаточно, но, как вы видели выше, когда была напечатана вся таблица, невозможно сказать, есть ли пропущенные записи, просто просмотрев предварительный просмотр записей вручную, особенно в случаях, когда набор данных огромен. Нам нужен более надежный способ достижения этой цели. Давайте приступим!

In [3]:
X.isnull().apply(lambda x: check_missing_values(x))

,text,category,category_name
0,The amoung of missing records is:,The amoung of missing records is:,The amoung of missing records is:
1,0,0,0


Хорошо, в этой одной строке кода произошло много всего, так что давайте разберем ее. Во-первых, с помощью `isnull` мы преобразовали нашу таблицу в таблицу **True/False**, которую вы видите выше, где **True** в этом случае означает, что данные отсутствуют, а **False** означает, что данные присутствуют. Затем мы берем преобразованную таблицу и применяем функцию к каждой строке, которая, по сути, подсчитывает, есть ли отсутствующие значения в каждой записи, и распечатывает, сколько отсутствующих значений мы нашли. Другими словами, функция `check_missing_values` просматривает каждое поле (атрибут или столбец) в наборе данных и подсчитывает, сколько пропущенных значений было найдено.

Есть много других умных способов проверить наличие отсутствующих данных, и именно это делает Pandas таким прекрасным инструментом для работы. Вы получаете контроль, который вам нужен как специалисту по анализу данных или просто человеку, работающему над проектами интеллектуального анализа данных. Действительно, Pandas облегчает вам жизнь!

---

### >>> **Упражнение 4:**
Давайте попробуем что-нибудь другое. Вместо вычисления отсутствующих значений по столбцам, давайте попробуем вычислить отсутствующие значения в каждой записи вместо каждого столбца.

**Подсказка**: параметр `axis`. Ознакомьтесь с [документацией](http://pandas.geekwriter.ru/getting_started/intro_tutorials/index.html) для получения дополнительной информации.

In [16]:
# Ответьте здесь
X.isnull().apply(lambda x: check_missing_values(x), axis=1)

,0
0,"(The amoung of missing records is: , 0)"
1,"(The amoung of missing records is: , 0)"
2,"(The amoung of missing records is: , 0)"
3,"(The amoung of missing records is: , 0)"
4,"(The amoung of missing records is: , 0)"
...,...
2252,"(The amoung of missing records is: , 0)"
2253,"(The amoung of missing records is: , 0)"
2254,"(The amoung of missing records is: , 0)"
2255,"(The amoung of missing records is: , 0)"


---

У нас есть функция для проверки отсутствующих записей, теперь давайте сделаем что-нибудь озорное и вставим некоторые фиктивные данные во фрейм данных и проверим надежность нашей функции. Эти фиктивные данные предназначены для повреждения набора данных. Я имею в виду, что это происходит сегодня довольно часто, особенно когда хакеры хотят угнать или повредить базу данных.

Мы вставим `Series`, которая по сути является «одномерным помеченным массивом, способным хранить данные любого типа (целые числа, строки, числа с плавающей запятой, объекты python и т. д.). Метки оси в совокупности называются индексом», в наш текущий фрейм данных.

In [4]:
dummy_series = pd.Series(["dummy_record", 1], index=["text", "category"])

In [5]:
dummy_series

,0
text,dummy_record
category,1


In [6]:
result_with_series = pd.concat([X, dummy_series])

In [7]:
# проверка, была ли запись зафиксирована в результате
len(result_with_series)

2259

Теперь, когда мы добавили запись с некоторыми пропущенными значениями. Давайте попробуем нашу функцию и посмотрим, сможет ли она обнаружить, что в результирующем фрейме данных есть пропущенное значение.

In [8]:
result_with_series.isnull().apply(lambda x: check_missing_values(x))

,text,category,category_name,0
0,The amoung of missing records is:,The amoung of missing records is:,The amoung of missing records is:,The amoung of missing records is:
1,2,2,2,2257


Действительно, в этом новом фрейме данных есть пропущенное значение. В частности, пропущенное значение происходит из атрибута `category_name`. Как я упоминал ранее, есть много способов выполнять определенные операции с фреймами данных. В этом случае давайте используем простой словарь и попробуем вставить его в наш исходный фрейм данных `X`. Обратите внимание, что выше мы не меняем фрейм данных `X`, поскольку результаты применяются непосредственно к предоставленной переменной присваивания. Но в случае, если мы просто хотим упростить задачу, мы можем просто применить изменения непосредственно к `X` и присвоить его себе, как мы это сделаем ниже. Эта модификация создаст необходимость удалить эту фиктивную запись позже, что означает, что нам нужно узнать больше о фреймах данных Pandas. Все становится напряженным! Но просто расслабьтесь, все будет хорошо!

In [9]:
# фиктивная запись в формате словаря
dummy_dict = [{'text': 'dummy_record',
               'category': 1
              }]

In [10]:
X = pd.concat( [X, pd.DataFrame.from_dict(dummy_dict)])

In [11]:
len(X)

2258

In [12]:
X.isnull().apply(lambda x: check_missing_values(x))

,text,category,category_name
0,The amoung of missing records is:,The amoung of missing records is:,The amoung of missing records is:
1,0,0,1


Итак, теперь, когда мы видим, что в наших данных есть пропущенные значения, мы хотим удалить записи с пропущенными значениями. Код для удаления записи с пропущенными значениями, которую мы только что добавили, выглядит следующим образом:

In [13]:
X.dropna(inplace=True)

... а теперь давайте проверим, избавились ли мы от записей с пропущенными значениями.

In [14]:
X.isnull().apply(lambda x: check_missing_values(x))

,text,category,category_name
0,The amoung of missing records is:,The amoung of missing records is:,The amoung of missing records is:
1,0,0,0


In [15]:
len(X)

2257

И мы вернулись к нашему исходному набору данных, чистому и опрятному, как мы и хотели. Этого достаточно о том, как бороться с пропущенными значениями, давайте теперь перейдем к чему-то более интересному.

Но на случай, если вы хотите узнать больше о том, как работать с недостающими данными, прочитайте эту [статью на habr](https://habr.com/ru/articles/773214/).

---

### >>> **Упражнение 5**
Есть старая поговорка: «Дьявол кроется в деталях». Когда мы работаем с чрезвычайно большими данными, трудно проверять записи одну за другой (как мы это делали до сих пор). И кроме того, мы даже не знаем, с какими видами пропущенных значений мы сталкиваемся. Таким образом, навыки «отладки» становятся более отточенными, чем больше времени мы тратим на решение ошибок. Давайте сосредоточимся на другом методе проверки отсутствующих значений и типах отсутствующих значений, с которыми вы можете столкнуться. Проверить наличие отсутствующих значений не так-то просто, как вы сейчас узнаете.

Пожалуйста, проверьте данные и процесс ниже, опишите, что вы наблюдаете и почему это произошло.
$Hint$ : почему `.isnull()` не сработал?

In [17]:
import numpy as np

NA_dict = [{ 'id': 'A', 'missing_example': np.nan },
           { 'id': 'B'                    },
           { 'id': 'C', 'missing_example': 'NaN'  },
           { 'id': 'D', 'missing_example': 'None' },
           { 'id': 'E', 'missing_example':  None  },
           { 'id': 'F', 'missing_example': ''     }]

NA_df = pd.DataFrame(NA_dict, columns = ['id','missing_example'])
NA_df

,id,missing_example
0,A,NaN
1,B,NaN
2,C,NaN
3,D,None
4,E,None
5,F,


In [18]:
NA_df['missing_example'].isnull()

,missing_example
0,True
1,True
2,False
3,False
4,True
5,False


In [20]:
# Ответьте здесь
# Заменяем строки 'NaN', 'None', и '' на np.nan
NA_df['missing_example'] = NA_df['missing_example'].replace(['NaN', 'None', ''], np.nan)

NA_df['missing_example'].isnull() # Теперь isnull() обнаружит все, что мы считаем пропущенными значениями

,missing_example
0,True
1,True
2,True
3,True
4,True
5,True


---

### 4.2 Работа с дублирующимися данными
Работа с дублирующимися данными так же болезненна, как и работа с недостающими данными. Худший случай - это когда у вас есть дублирующиеся данные с отсутствующими значениями. Но давайте не будем увлекаться. Давайте придерживаться основ. Как мы узнали на нашем курсе Data Mining, дублирование данных может возникать по многим причинам. В большинстве случаев это связано с тем, как мы храним данные или как мы собираем и объединяем данные. Например, мы могли собрать и сохранить твит, а ретвит того же твита как две разные записи; это приводит к случаю дублирования данных; единственная разница в том, что один - это исходный твит, а другой - ретвит. Здесь вы узнаете, что работа с дублирующимися данными не так сложна, как с отсутствующими значениями. Но это также зависит от того, что вы считаете дублирующимися данными, т. е. все зависит от ваших критериев того, что считается дублирующейся записью, а также от типа данных, с которыми вы имеете дело. Для текстовых данных это может быть не так тривиально, как для числовых значений или изображений. В любом случае, давайте посмотрим на некоторый код о том, как работать с дублирующимися записями в нашем фрейме данных `X`.

Сначала давайте проверим, сколько дубликатов у нас есть в нашем текущем наборе данных. Вот строка кода, которая проверяет наличие дубликатов; она очень похожа на функцию `isnull`, которую мы использовали для проверки отсутствующих значений.

In [21]:
X.duplicated()

,0
0,False
1,False
2,False
3,False
4,False
...,...
2252,False
2253,False
2254,False
2255,False


Мы также можем проверить сумму дублирующихся записей, просто сделав:

In [22]:
sum(X.duplicated())

0

Основываясь на этом выводе, вы можете спросить, почему операция `duplicated` вернула только один столбец, который указывает, есть ли дублирующаяся запись или нет. Итак, да, все, что делает операция `duplicated()`, - это проверка для каждой записи вместо каждого столбца. Вот почему операция возвращает только одно значение вместо трех значений для каждого столбца. Похоже, у нас нет никаких дубликатов, поскольку ни одна из наших записей не привела к `True`. Если мы хотим проверить наличие дубликатов, как мы это делали выше для некоторого конкретного столбца, а не для всех столбцов, мы делаем что-то, как показано ниже. Как вы могли заметить, в случае, когда мы выбираем некоторые столбцы вместо проверки по всем столбцам, мы как бы снижаем критерии того, что считается дублирующейся записью. Итак, давайте проверим наличие дубликатов, только проверив атрибут `text`.

In [23]:
sum(X.duplicated('text'))

0

Теперь давайте создадим несколько дублирующихся фиктивных записей и добавим их в основной фрейм данных `X`. Затем давайте попробуем избавиться от дубликатов.

In [24]:
dummy_duplicate_dict = [{
                             'text': 'dummy record',
                             'category': 1,
                             'category_name': "dummy category"
                        },
                        {
                             'text': 'dummy record',
                             'category': 1,
                             'category_name': "dummy category"
                        }]

In [25]:
X = pd.concat( [X, pd.DataFrame.from_dict(dummy_duplicate_dict)])


In [26]:
len(X)

2259

In [27]:
sum(X.duplicated('text'))

1

Мы добавили фиктивные дубликаты в `X`. Теперь мы столкнулись с решением, что делать с дублирующимися записями после того, как мы их нашли. В нашем случае мы хотим избавиться от всех дублирующихся записей, не сохраняя копию. Мы можем просто сделать это с помощью следующей строки кода:

In [28]:
X.drop_duplicates(keep=False, inplace=True) # inplace применяет изменения непосредственно к нашему фрейму данных

In [29]:
len(X)

2257

Ознакомьтесь с [документацией](http://pandas.pydata.org/pandas-docs/stable/indexing.html?highlight=duplicate#duplicate-data) Pandas для получения дополнительной информации о работе с дублирующимися данными.

---